In [1]:
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn import metrics
import tensorflow as tf
import pandas as pd
import numpy as np
import plotting
import hls4ml
import shap
import sys
import os

np.random.seed(77)

import matplotlib.pyplot as plt
import mplhep
plt.style.use(mplhep.style.CMS)


def print_dict(d, indent=0):
    align=20
    for key, value in d.items():
        print('  ' * indent + str(key), end='')
        if isinstance(value, dict):
            print()
            print_dict(value, indent+1)
        else:
            print(':' + ' ' * (20 - len(key) - 2 * indent) + str(value))

2023-07-08 08:39:28.922044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/exp_soft/llr/python/3.9.9/lib/vtk:/opt/exp_soft/llr/python/3.9.9/lib:/usr/lib64/classads:/usr/lib64:/usr/lib
2023-07-08 08:39:28.922073: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/llr/cms/motta/.local/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes,

In [2]:
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/settings64.sh
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/settings64.sh

In [3]:
options = {
    'v'          : '22QNTZD',
    'date'       : '2023_07_06',
    'caloClNxM'  : '5x9',
    'sparsityID'   : 0.25,
    'sparsityCAL'   : 0.5
}

In [4]:
indir = '/data_CMS/cms/motta/Phase2L1T/'+options['date']+'_v'+options['v']+'/TauMinator_CB_cltw'+options['caloClNxM']+'_Training'

N = int(options['caloClNxM'].split('x')[0])
M = int(options['caloClNxM'].split('x')[1])

sparsityTag_id = '_QNTZD_sparsity'+str(options['sparsityID'])
sparsityTag_cal = '_QNTZD_sparsity'+str(options['sparsityCAL'])

**Only models with one input tensor are currently supported by VivadoAcceleratorBackend.
Therefore, the CNN model is in this case a special version that takes only the image as an input and gives as an output the flattened tensor out of the convolutions withou concatenating the position of the cluster.**

In [5]:
# quantized and pruned models
QCNN = keras.models.load_model(indir+'/CNNmodel'+sparsityTag_id+'_special4accelerator', compile=False)
QDNNident = keras.models.load_model(indir+'/ID_DNNmodel'+sparsityTag_id, compile=False)
QDNNcalib = keras.models.load_model(indir+'/CAL_DNNmodel'+sparsityTag_cal, compile=False)

2023-07-08 08:39:36.226088: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /grid_mnt/opt__exp_soft/llr/python/3.9.9/lib/python3.9/site-packages/cv2/../../lib64:/opt/exp_soft/llr/python/3.9.9/lib/vtk:/opt/exp_soft/llr/python/3.9.9/lib:/usr/lib64/classads:/usr/lib64:/usr/lib
2023-07-08 08:39:36.226133: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-08 08:39:36.226155: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (llrgrhgtrig.in2p3.fr): /proc/driver/nvidia/version does not exist
2023-07-08 08:39:36.226555: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical

# QUANTIZED MODELS ONLY

In [6]:
############################## Pass quantized CNN model through hls4ml ##############################

compl = True
synth = True
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    QCNN_hls_cfg = hls4ml.utils.config_from_keras_model(QCNN, granularity='name')
    QCNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    QCNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in QCNN_hls_cfg['LayerName'].keys():
        QCNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
        QCNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        QCNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    QCNN_hls_cfg['LayerName']['TowerClusterImage']['Precision'] = 'ap_ufixed<10,8>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['ParallelizationFactor'] = 4
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<14,7>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<14,7>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<10,7>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<10,7>'

    QCNN_hls_cfg['LayerName']['MP_CNNpBNlayer1']['Precision'] = 'ap_fixed<10,7>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<14,6>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<14,6>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<9,6>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<9,6>'

    print(QCNN_hls_cfg)

    QCNN_hls = hls4ml.converters.convert_from_keras_model(QCNN,
                                                          hls_config=QCNN_hls_cfg,
                                                          output_dir=indir+'/CNNmodel_HLS_XCU200deployment/',
                                                          backend='VivadoAccelerator',
                                                          board='alveo-u200',
                                                          io_type='io_parallel',
                                                          platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QCNN_hls.compile()

if synth:
    QCNN_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=True)

Interpreting Model
Topology:
Layer name: TowerClusterImage, layer type: Input
Layer name: CNNpBNlayer1, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer1
Layer name: RELU_CNNpBNlayer1, layer type: QActivation
  -> Activation (quantized_relu(10,7)), layer name: RELU_CNNpBNlayer1
Layer name: MP_CNNpBNlayer1, layer type: MaxPooling2D
Layer name: CNNpBNlayer2, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer2
Layer name: RELU_CNNpBNlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CNNpBNlayer2
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'TowerClusterImage': {'Precision': 'ap_ufixed<10,8>', 'Strategy': 'Latency', 'ReuseFactor': 1, 'Trace': True}, 'CNNpBNlayer1': {'Precision': {'weight': 'ap_fixed<5,1>', 'bias': 'ap_fixed<6,1>', 'accum': 'ap_fixed<14,7>', 'result': 'ap_fixed<14,7>'}, 'ReuseFactor': 1, 'Strategy': 'Latency', 'Trace': True, 'Pa

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'Product2' (firmware/nnet_utils/nnet_conv2d_latency.h:210) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 4.
INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_conv2d_latency.h:216) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 4.
INFO: [HLS 200-489] Unrolling loop 'Accum1' (firmware/nnet_utils/nnet_conv2d_latency.h:221) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Accum2' (firmware/nnet_utils/nnet_conv2d_latency.h:222) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3,

INFO: [XFORM 203-11] Balancing expressions in function 'nnet::fill_buffer_2<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>::fill_buffer' (firmware/nnet_utils/nnet_code_gen.h:38)...13 expression(s) balanced.
INFO: [XFORM 203-11] Balancing expressions in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (firmware/nnet_utils/nnet_conv2d_latency.h:75:59)...112 expression(s) balanced.
INFO: [XFORM 203-11] Balancing expressions in function 'nnet::conv_2d_cl<ap_fixed<10, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' (firmware/nnet_utils/nnet_conv2d_latency.h:75:59)...62 expression(s) balanced.
INFO: [HLS 200-111] Finished Pre-synthesis Time (s): cpu = 00:00:45 ; elapsed = 00:00:50 . Memory (MB): peak = 1824.188 ; gain = 1295.879 ; free physical = 51130 ; free virtual = 339945
INFO: [HLS 200-111] Finished Architecture Synthesis Time (s): cpu = 00:00:57 ;

INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.24 seconds; current allocated memory: 610.934 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'conv_2d_cl_ap_fixed_ap_fixed_14_6_5_3_0_config5_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining loop 'PartitionLoop'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 2.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 1.15 seconds; current allocated memory: 612.990 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIN

...

Synthesis Optimization Runtime : Time (s): cpu = 00:01:40 ; elapsed = 00:03:14 . Memory (MB): peak = 3379.125 ; gain = 1119.215 ; free physical = 48545 ; free virtual = 337659
Synthesis Optimization Complete : Time (s): cpu = 00:01:40 ; elapsed = 00:03:14 . Memory (MB): peak = 3379.125 ; gain = 1119.215 ; free physical = 48552 ; free virtual = 337659
INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.23 ; elapsed = 00:00:00.24 . Memory (MB): peak = 3379.336 ; gain = 0.000 ; free physical = 48545 ; free virtual = 337651
INFO: [Netlist 29-17] Analyzing 1433 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00.01 ; elapsed = 00:00:00.01 . Memory (MB): peak = 3438.262 ; gain = 0.000 ; free physical = 48460 ; free virtu

INFO: [IP_Flow 19-2181] Payment Required is not set for this core.
INFO: [IP_Flow 19-2187] The Product Guide file is missing.
INFO: [Ipptcl 7-1486] check_integrity: Integrity check passed.
INFO: [Common 17-206] Exiting Vivado at Sat Jul  8 08:49:00 2023...
Option Map File Used: '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/data/vitis/vpp/optMap.xml'

****** v++ v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/reports/link
	Log files: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/logs/link
INFO: [v++ 60-1657] Initializing dispatch client.
Running Dispat

INFO: [CF2BD 82-28] cf2xd finished successfully
INFO: [CF2BD 82-31] Launching cf_xsd: cf_xsd -disable-address-gen -bd pfm_dynamic.bd -dn dr -dp /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/sys_link/_sysl/.xsd
INFO: [CF2BD 82-28] cf_xsd finished successfully
INFO: [SYSTEM_LINK 82-37] [08:49:44] cf2bd finished successfully
Time (s): cpu = 00:00:02 ; elapsed = 00:00:02 . Memory (MB): peak = 1402.066 ; gain = 0.000 ; free physical = 51414 ; free virtual = 340259
INFO: [v++ 60-1441] [08:49:44] Run run_link: Step system_link: Completed
Time (s): cpu = 00:00:12 ; elapsed = 00:00:14 . Memory (MB): peak = 1436.730 ; gain = 0.000 ; free physical = 51443 ; free virtual = 340284
INFO: [v++ 60-1443] [08:49:44] Run run_link: Step cf2sw: Started
INFO: [v++ 60-1453] Command Line: cf2sw -sdsl /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CNNmodel_

[09:09:21] Top-level synthesis in progress.
[09:09:34] Run vpl: Step synth: Completed
[09:09:34] Run vpl: Step impl: Started
[09:19:53] Finished 2nd of 6 tasks (FPGA linking synthesized kernels to platform). Elapsed time: 00h 30m 04s 

[09:19:53] Starting logic optimization..
[09:21:26] Phase 1 Retarget
[09:21:57] Phase 2 Constant propagation
[09:21:57] Phase 3 Sweep
[09:22:27] Phase 4 BUFG optimization
[09:22:58] Phase 5 Shift Register Optimization
[09:22:58] Phase 6 Post Processing Netlist
[09:24:00] Finished 3rd of 6 tasks (FPGA logic optimization). Elapsed time: 00h 04m 07s 

[09:24:00] Starting logic placement..
[09:25:02] Phase 1 Placer Initialization
[09:25:02] Phase 1.1 Placer Initialization Netlist Sorting
[09:26:35] Phase 1.2 IO Placement/ Clock Placement/ Build Placer Device
[09:27:36] Phase 1.3 Build Placer Netlist Model
[09:29:40] Phase 1.4 Constrain Clocks/Macros
[09:29:40] Phase 2 Global Placement
[09:29:40] Phase 2.1 Floorplanning
[09:30:11] Phase 2.1.1 Partition Driven

XRT Build Version: 2.6.0 (Vitis)
       Build Date: 2020-05-07 15:30:09
          Hash ID: 9d35aca9e6be09a5402ec036a5607d26e74e01cc
Creating a default 'in-memory' xclbin image.

Section: 'DEBUG_IP_LAYOUT'(9) was successfully added.
Size   : 440 bytes
Format : JSON
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/debug_ip_layout.rtd'

Section: 'BITSTREAM'(0) was successfully added.
Size   : 42312850 bytes
Format : RAW
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/partial.bit'

Section: 'MEM_TOPOLOGY'(6) was successfully added.
Format : JSON
File   : 'mem_topology'

Section: 'IP_LAYOUT'(8) was successfully added.
Format : JSON
File   : 'ip_layout'

Section: 'CONNECTIVITY'(7) was successfully added.
Format : JSON
File   : 'connectivity'

Section: 'CLOCK_FREQ_T

In [7]:
############################## Pass quantized identification DNN model through hls4ml ##############################

compl = True
synth = True
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    id_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNident, granularity='name')
    id_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    id_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in id_QDNN_hls_cfg['LayerName'].keys():
        id_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        id_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        id_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    id_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['accum'] = 'ap_fixed<13,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['result'] = 'ap_fixed<10,6>'
    # id_QDNN_hls_cfg['LayerName']['RELU_IDlayer1']['Precision'] = 'ap_fixed<13,3>'

    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['accum'] = 'ap_fixed<11,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['result'] = 'ap_fixed<10,4>'
    id_QDNN_hls_cfg['LayerName']['RELU_IDlayer2']['Precision'] = 'ap_fixed<7,3>'

    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['weight'] = 'ap_fixed<2,1>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['accum'] = 'ap_fixed<7,4>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['result'] = 'ap_fixed<7,4>'

    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Precision'] = 'ap_fixed<8,1>'
    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Strategy'] = 'Stable'

    print(id_QDNN_hls_cfg)

    QDNNident_hls = hls4ml.converters.convert_from_keras_model(QDNNident,
                                                               hls_config=id_QDNN_hls_cfg,
                                                               output_dir=indir+'/ID_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_parallel',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNident_hls.compile()
    
if synth:
    QDNNident_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=True)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: IDlayer1, layer type: QDense
  -> Activation (linear), layer name: IDlayer1
Layer name: RELU_IDlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,1)), layer name: RELU_IDlayer1
Layer name: IDlayer2, layer type: QDense
  -> Activation (linear), layer name: IDlayer2
Layer name: RELU_IDlayer2, layer type: QActivation
  -> Activation (quantized_relu(8,1)), layer name: RELU_IDlayer2
Layer name: IDout, layer type: QDense
  -> Activation (linear), layer name: IDout
Layer name: sigmoid_IDout, layer type: Activation
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'IDlayer1': {'Precision': {'weight': 'ap_fixed<3,1>', 'accum': 'ap_fixed<13,6>', 'result': 'ap_fixed<10,6>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'IDlayer1_linear': 

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_dense_latency.h:68) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'Accum1' (firmware/nnet_utils/nnet_dense_latency.h:73) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 26.
INFO: [HLS 200-489] Unrolling loop 'Accum2' (firmware/nnet_utils/nnet_dense_latency.h:74) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_latency.h:81) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0.0.0' by setting 'biases[5].V' to 'b2.V.5'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0.0.0.0' by setting 'biases[6].V' to 'b2.V.6'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config

INFO: [HLS 200-111] Finished Architecture Synthesis Time (s): cpu = 00:00:49 ; elapsed = 00:00:52 . Memory (MB): peak = 1824.188 ; gain = 1295.879 ; free physical = 51396 ; free virtual = 339848
INFO: [HLS 200-10] Starting hardware synthesis ...
INFO: [HLS 200-10] Synthesizing 'myproject_axi' ...
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'dense_latency_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining function 'dense_latency.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 2.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 53.05 seconds; current allocated memory: 472.859 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performi

...

INFO: [Common 17-206] Exiting Vivado at Sat Jul  8 10:37:26 2023...
***** VIVADO SYNTHESIS COMPLETED IN 0h2m24s *****
INFO: [HLS 200-112] Total elapsed time: 262.63 seconds; peak allocated memory: 555.377 MB.
INFO: [Common 17-206] Exiting vivado_hls at Sat Jul  8 10:37:37 2023...
Cosim report not found.
Timing report not found.

****** Vivado v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
  **** IP Build 2902112 on Wed May 27 22:43:36 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

source design.tcl
# set tcldir [file dirname [info script]]
# source [file join $tcldir project.tcl]
## variable myproject
## set myproject "myproject"
## variable backend
## set backend vivadoaccelerator
## variable part
## set part "xcu200-fsgd2104-2-e"
## set bit_width_hls_output 32
## set bit_width_hls_input 32
# create_project project_1 ${myproject}_vivado_accelerator -part ${part} -force
# set_property  ip_repo_paths  ${myproject}_prj [current_project]
#

INFO: [IP_Flow 19-2181] Payment Required is not set for this core.
INFO: [IP_Flow 19-2187] The Product Guide file is missing.
INFO: [Ipptcl 7-1486] check_integrity: Integrity check passed.
INFO: [Common 17-206] Exiting Vivado at Sat Jul  8 10:38:01 2023...
Option Map File Used: '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/data/vitis/vpp/optMap.xml'

****** v++ v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/reports/link
	Log files: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/logs/link
INFO: [v++ 60-1657] Initializing dispatch client.
Running 

INFO: [CF2BD 82-28] cf2xd finished successfully
INFO: [CF2BD 82-31] Launching cf_xsd: cf_xsd -disable-address-gen -bd pfm_dynamic.bd -dn dr -dp /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/sys_link/_sysl/.xsd
INFO: [CF2BD 82-28] cf_xsd finished successfully
INFO: [SYSTEM_LINK 82-37] [10:38:45] cf2bd finished successfully
Time (s): cpu = 00:00:02 ; elapsed = 00:00:02 . Memory (MB): peak = 1402.066 ; gain = 0.000 ; free physical = 51795 ; free virtual = 340260
INFO: [v++ 60-1441] [10:38:45] Run run_link: Step system_link: Completed
Time (s): cpu = 00:00:12 ; elapsed = 00:00:13 . Memory (MB): peak = 1436.730 ; gain = 0.000 ; free physical = 51825 ; free virtual = 340285
INFO: [v++ 60-1443] [10:38:45] Run run_link: Step cf2sw: Started
INFO: [v++ 60-1453] Command Line: cf2sw -sdsl /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/ID_DNN

[11:09:55] Phase 1.2 IO Placement/ Clock Placement/ Build Placer Device
[11:11:27] Phase 1.3 Build Placer Netlist Model
[11:13:01] Phase 1.4 Constrain Clocks/Macros
[11:13:01] Phase 2 Global Placement
[11:13:01] Phase 2.1 Floorplanning
[11:13:32] Phase 2.1.1 Partition Driven Placement
[11:13:32] Phase 2.1.1.1 PBP: Partition Driven Placement
[11:14:03] Phase 2.1.1.2 PBP: Clock Region Placement
[11:14:03] Phase 2.1.1.3 PBP: Compute Congestion
[11:14:34] Phase 2.1.1.4 PBP: UpdateTiming
[11:14:34] Phase 2.1.1.5 PBP: Add part constraints
[11:14:34] Phase 2.2 Global Placement Core
[11:20:46] Phase 2.2.1 Physical Synthesis In Placer
[11:22:20] Phase 3 Detail Placement
[11:22:20] Phase 3.1 Commit Multi Column Macros
[11:22:20] Phase 3.2 Commit Most Macros & LUTRAMs
[11:22:51] Phase 3.3 Area Swap Optimization
[11:22:51] Phase 3.4 Pipeline Register Optimization
[11:22:51] Phase 3.5 IO Cut Optimizer
[11:23:53] Phase 3.6 Fast Optimization
[11:24:24] Phase 3.7 Small Shape DP
[11:24:24] Phase 3.7.1 

XRT Build Version: 2.6.0 (Vitis)
       Build Date: 2020-05-07 15:30:09
          Hash ID: 9d35aca9e6be09a5402ec036a5607d26e74e01cc
Creating a default 'in-memory' xclbin image.

Section: 'DEBUG_IP_LAYOUT'(9) was successfully added.
Size   : 440 bytes
Format : JSON
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/debug_ip_layout.rtd'

Section: 'BITSTREAM'(0) was successfully added.
Size   : 41266850 bytes
Format : RAW
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/partial.bit'

Section: 'MEM_TOPOLOGY'(6) was successfully added.
Format : JSON
File   : 'mem_topology'

Section: 'IP_LAYOUT'(8) was successfully added.
Format : JSON
File   : 'ip_layout'

Section: 'CONNECTIVITY'(7) was successfully added.
Format : JSON
File   : 'connectivity'

Section: 'CLOCK_

In [8]:
############################## Pass quantized calibration DNN model through hls4ml ##############################

compl = True
synth = True
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    cal_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNcalib, granularity='name')
    cal_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    cal_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in cal_QDNN_hls_cfg['LayerName'].keys():
        cal_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        cal_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        cal_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    cal_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['weight'] = 'ap_fixed<6,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['accum'] = 'ap_fixed<16,5>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['result'] = 'ap_fixed<16,7>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer1']['Precision'] = 'ap_fixed<11,7>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['accum'] = 'ap_fixed<16,7>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['result'] = 'ap_fixed<12,8>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer2']['Precision'] = 'ap_fixed<10,7>'

    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['weight'] = 'ap_fixed<2,1>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['accum'] = 'ap_fixed<12,9>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['result'] = 'ap_fixed<11,10>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Strategy'] = 'Stable'

    print(cal_QDNN_hls_cfg)

    QDNNcalib_hls = hls4ml.converters.convert_from_keras_model(QDNNcalib,
                                                               hls_config=cal_QDNN_hls_cfg,
                                                               output_dir=indir+'/CAL_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_parallel',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNcalib_hls.compile()

if synth:
    QDNNcalib_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=True)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: CALlayer1, layer type: QDense
  -> Activation (linear), layer name: CALlayer1
Layer name: RELU_CALlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer1
Layer name: CALlayer2, layer type: QDense
  -> Activation (linear), layer name: CALlayer2
Layer name: RELU_CALlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer2
Layer name: CALout, layer type: QDense
  -> Activation (linear), layer name: CALout
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'CALlayer1': {'Precision': {'weight': 'ap_fixed<6,1>', 'accum': 'ap_fixed<16,5>', 'result': 'ap_fixed<16,7>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'CALlayer1_linear': {'Precision': 'ap_fixed<16,6>', 'Reuse

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_dense_latency.h:68) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 64.
INFO: [HLS 200-489] Unrolling loop 'Accum1' (firmware/nnet_utils/nnet_dense_latency.h:73) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 26.
INFO: [HLS 200-489] Unrolling loop 'Accum2' (firmware/nnet_utils/nnet_dense_latency.h:74) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 64.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_latency.h:81) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 7, (ap_q_mode)5, (ap_o_mode)3

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0' at call site (firmware/nnet_utils/nnet_dense.h:46->firmware/myproject.cpp:65) by setting 'biases[3].V' to 'b4.V.3'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0' at call site (firmware/nnet_utils/nnet_dense.h:46->firmware/myproject.cpp:65) by setting 'biases[4].V' to 'b4.V.4'.
INFO: [XFORM 203-622] Instan

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0' at call site (firmware/nnet_utils/nnet_dense.h:46->firmware/myproject.cpp:65) by setting 'biases[19].V' to 'b4.V.19'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0' at call site (

...

Synthesis Optimization Runtime : Time (s): cpu = 00:01:24 ; elapsed = 00:02:50 . Memory (MB): peak = 3136.105 ; gain = 876.195 ; free physical = 41859 ; free virtual = 330554
Synthesis Optimization Complete : Time (s): cpu = 00:01:24 ; elapsed = 00:02:50 . Memory (MB): peak = 3136.105 ; gain = 876.195 ; free physical = 41872 ; free virtual = 330559
INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.47 ; elapsed = 00:00:00.49 . Memory (MB): peak = 3144.324 ; gain = 0.000 ; free physical = 41827 ; free virtual = 330514
INFO: [Netlist 29-17] Analyzing 4120 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 3 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00 ; elapsed = 00:00:00.01 . Memory (MB): peak = 3422.215 ; gain = 0.000 ; free physical = 41624 ; free virtual = 

INFO: [IP_Flow 19-2181] Payment Required is not set for this core.
INFO: [IP_Flow 19-2187] The Product Guide file is missing.
INFO: [Ipptcl 7-1486] check_integrity: Integrity check passed.
INFO: [Common 17-206] Exiting Vivado at Sat Jul  8 12:28:55 2023...
Option Map File Used: '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/data/vitis/vpp/optMap.xml'

****** v++ v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/reports/link
	Log files: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/logs/link
INFO: [v++ 60-1657] Initializing dispatch client.
Runnin

INFO: [CF2BD 82-28] cf2xd finished successfully
INFO: [CF2BD 82-31] Launching cf_xsd: cf_xsd -disable-address-gen -bd pfm_dynamic.bd -dn dr -dp /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/sys_link/_sysl/.xsd
INFO: [CF2BD 82-28] cf_xsd finished successfully
INFO: [SYSTEM_LINK 82-37] [12:29:39] cf2bd finished successfully
Time (s): cpu = 00:00:02 ; elapsed = 00:00:02 . Memory (MB): peak = 1402.066 ; gain = 0.000 ; free physical = 51684 ; free virtual = 340259
INFO: [v++ 60-1441] [12:29:39] Run run_link: Step system_link: Completed
Time (s): cpu = 00:00:12 ; elapsed = 00:00:13 . Memory (MB): peak = 1436.730 ; gain = 0.000 ; free physical = 51712 ; free virtual = 340282
INFO: [v++ 60-1443] [12:29:39] Run run_link: Step cf2sw: Started
INFO: [v++ 60-1453] Command Line: cf2sw -sdsl /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CAL_D

[12:46:23] Block-level synthesis in progress, 7 of 8 jobs complete, 1 job running.
[12:46:54] Block-level synthesis in progress, 7 of 8 jobs complete, 1 job running.
[12:47:25] Block-level synthesis in progress, 7 of 8 jobs complete, 1 job running.
[12:47:56] Block-level synthesis in progress, 7 of 8 jobs complete, 1 job running.
[12:48:27] Block-level synthesis in progress, 7 of 8 jobs complete, 1 job running.
[12:48:58] Block-level synthesis in progress, 7 of 8 jobs complete, 1 job running.
[12:49:29] Top-level synthesis in progress.
[12:50:00] Top-level synthesis in progress.
[12:50:31] Top-level synthesis in progress.
[12:51:01] Top-level synthesis in progress.
[12:51:32] Top-level synthesis in progress.
[12:52:03] Top-level synthesis in progress.
[12:52:33] Top-level synthesis in progress.
[12:53:04] Top-level synthesis in progress.
[12:53:24] Run vpl: Step synth: Completed
[12:53:24] Run vpl: Step impl: Started
[13:04:42] Finished 2nd of 6 tasks (FPGA linking synthesized kernels 

XRT Build Version: 2.6.0 (Vitis)
       Build Date: 2020-05-07 15:30:09
          Hash ID: 9d35aca9e6be09a5402ec036a5607d26e74e01cc
Creating a default 'in-memory' xclbin image.

Section: 'DEBUG_IP_LAYOUT'(9) was successfully added.
Size   : 440 bytes
Format : JSON
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/debug_ip_layout.rtd'

Section: 'BITSTREAM'(0) was successfully added.
Size   : 43676354 bytes
Format : RAW
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CB_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/partial.bit'

Section: 'MEM_TOPOLOGY'(6) was successfully added.
Format : JSON
File   : 'mem_topology'

Section: 'IP_LAYOUT'(8) was successfully added.
Format : JSON
File   : 'ip_layout'

Section: 'CONNECTIVITY'(7) was successfully added.
Format : JSON
File   : 'connectivity'

Section: 'CLOC